In [34]:
import os
os.environ['OMP_NUM_THREADS'] = '2'
os.environ['export OPENBLAS_NUM_THREADS']='2'

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from utilities3 import *

import operator
from functools import reduce
from functools import partial

from timeit import default_timer

from Adam import Adam

import pickle

torch.manual_seed(0)
np.random.seed(0)

In [35]:
################################################################
# 3d fourier layers
################################################################

class SpectralConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2, modes3):
        super(SpectralConv3d, self).__init__()

        """
        3D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1 #Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2
        self.modes3 = modes3

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))
        self.weights3 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))
        self.weights4 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul3d(self, input, weights):
        # (batch, in_channel, x,y,t ), (in_channel, out_channel, x,y,t) -> (batch, out_channel, x,y,t)
        return torch.einsum("bixyz,ioxyz->boxyz", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfftn(x, dim=[-3,-2,-1])

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-3), x.size(-2), x.size(-1)//2 + 1, dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, :self.modes1, :self.modes2, :self.modes3], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, -self.modes1:, :self.modes2, :self.modes3], self.weights2)
        out_ft[:, :, :self.modes1, -self.modes2:, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, :self.modes1, -self.modes2:, :self.modes3], self.weights3)
        out_ft[:, :, -self.modes1:, -self.modes2:, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, -self.modes1:, -self.modes2:, :self.modes3], self.weights4)

        #Return to physical space
        x = torch.fft.irfftn(out_ft, s=(x.size(-3), x.size(-2), x.size(-1)))
        return x

class FNO3d(nn.Module):
    def __init__(self, modes1, modes2, modes3, width):
        super(FNO3d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the first 10 timesteps + 3 locations (u(1, x, y), ..., u(10, x, y),  x, y, t). It's a constant function in time, except for the last index.
        input shape: (batchsize, x=64, y=64, t=40, c=13)
        output: the solution of the next 40 timesteps
        output shape: (batchsize, x=64, y=64, t=40, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.modes3 = modes3
        self.width = width
        self.padding = 15 # pad the domain if input is non-periodic
        self.fc0 = nn.Linear(4, self.width)
        # input channel is 12: the solution of the first 10 timesteps + 3 locations (u(1, x, y), ..., u(10, x, y),  x, y, t)

        self.conv0 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.conv1 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.conv2 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.conv3 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.w0 = nn.Conv3d(self.width, self.width, 1)
        self.w1 = nn.Conv3d(self.width, self.width, 1)
        self.w2 = nn.Conv3d(self.width, self.width, 1)
        self.w3 = nn.Conv3d(self.width, self.width, 1)
        self.bn0 = torch.nn.BatchNorm3d(self.width)
        self.bn1 = torch.nn.BatchNorm3d(self.width)
        self.bn2 = torch.nn.BatchNorm3d(self.width)
        self.bn3 = torch.nn.BatchNorm3d(self.width)

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 4, 1, 2, 3)
        x = F.pad(x, [0,self.padding]) # pad the domain if input is non-periodic

        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        x = x[..., :-self.padding]
        x = x.permute(0, 2, 3, 4, 1) # pad the domain if input is non-periodic
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, shape, device):
        batchsize, size_x, size_y, size_z = shape[0], shape[1], shape[2], shape[3]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1, 1).repeat([batchsize, 1, size_y, size_z, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1, 1).repeat([batchsize, size_x, 1, size_z, 1])
        gridz = torch.tensor(np.linspace(0, 1, size_z), dtype=torch.float)
        gridz = gridz.reshape(1, 1, 1, size_z, 1).repeat([batchsize, size_x, size_y, 1, 1])
        return torch.cat((gridx, gridy, gridz), dim=-1).to(device)

In [36]:
################################################################
# configs
################################################################
DATA_PATH = 'inverse/friction_line.npy'

# currently data are 400 samples
ntrain = 300
ntest = 100

modes = 8
width = 20

batch_size = 10

epochs = 500
learning_rate = 0.001
scheduler_step = 100
scheduler_gamma = 0.5

print(epochs, learning_rate, scheduler_step, scheduler_gamma)

path = f'friction_line_ep{epochs}'
path_model = 'model/'+path
path_train_err = 'results/'+path+'train.txt'
path_test_err = 'results/'+path+'test.txt'
path_image = 'image/'+path

runtime = np.zeros(2, )
t1 = default_timer()

S1 = 65
S2 = 97
T_in = 1
T = 1
step = 1

500 0.001 100 0.5


In [37]:
################################################################
# load data
################################################################
data_gen = np.load(DATA_PATH)

# np.random.shuffle(data_gen)
train_a = torch.tensor(data_gen[:ntrain,:,:,:T_in], dtype=torch.float)
train_u = torch.tensor(data_gen[:ntrain,:,:,T_in:T+T_in], dtype=torch.float)

test_a = torch.tensor(data_gen[-ntest:,:,:,:T_in], dtype=torch.float)
test_u = torch.tensor(data_gen[-ntest:,:,:,T_in:T+T_in], dtype=torch.float)

print(train_a.shape)
print(test_u.shape)
assert (S1 == train_u.shape[-3])
assert (S2 == train_u.shape[-2])
assert (T == train_u.shape[-1])


a_normalizer = UnitGaussianNormalizer(train_a)
with open('a_normalizer_friction_line.pkl', 'wb') as f:
    pickle.dump(a_normalizer, f)

train_a = a_normalizer.encode(train_a)
test_a = a_normalizer.encode(test_a)

y_normalizer = UnitGaussianNormalizer(train_u)
with open('y_normalizer_friction_line.pkl', 'wb') as f:
    pickle.dump(y_normalizer, f)

train_u = y_normalizer.encode(train_u)

train_a = train_a.reshape(ntrain,S1,S2,1,T_in).repeat([1,1,1,T,1])
test_a = test_a.reshape(ntest,S1,S2,1,T_in).repeat([1,1,1,T,1])

# train_a = train_a.reshape(ntrain,S1,S2,T_in)
# test_a = test_a.reshape(ntest,S1,S2,T_in)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_a, train_u), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

t2 = default_timer()

print('preprocessing finished, time used:', t2-t1)
device = torch.device('cuda')

torch.Size([300, 65, 97, 1])
torch.Size([100, 65, 97, 1])
preprocessing finished, time used: 0.12813877302687615


In [38]:
################################################################
# training and evaluation
################################################################
model = FNO3d(modes, modes, modes, width).cuda()

print(count_params(model))
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)


myloss = LpLoss(size_average=False)
y_normalizer.cuda()
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x).view(batch_size, S1, S2, T)

        mse = F.mse_loss(out, y, reduction='mean')
        # mse.backward()

        y = y_normalizer.decode(y)
        out = y_normalizer.decode(out)
        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward()

        optimizer.step()
        train_mse += mse.item()
        train_l2 += l2.item()

    scheduler.step()

    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x).view(batch_size, S1, S2, T)
            out = y_normalizer.decode(out)
            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_mse /= len(train_loader)
    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_mse, train_l2, test_l2)
torch.save(model, path_model)

6558357
0 2.3021401719888672 0.9661902576684952 0.06381030986706415 0.05476492613554001
1 2.3025907530100085 0.2460849813185632 0.023007879455884298 0.01180118389427662
2 2.296365642978344 0.02673190695544084 0.00822261640181144 0.012746896222233772
3 2.30000938300509 0.0251544794999063 0.007322221547365188 0.009411156699061394
4 2.3037243929575197 0.021578277523318926 0.00595943504323562 0.009078604280948638
5 2.3080013819853775 0.024030363404502472 0.0060442976032694185 0.009915474727749824
6 2.309710672998335 0.021357631341864667 0.005166818362971147 0.008857761770486831
7 2.3078678219462745 0.02167406879986326 0.0051158714294433595 0.00846852209419012
8 2.31304601195734 0.022078937695672114 0.005146025630335013 0.00833653461188078
9 2.3123771020327695 0.021893352270126343 0.005001151598989963 0.008753578029572964
10 2.31527942197863 0.021211737828950087 0.004713930990546942 0.008043026663362979
11 2.313670882023871 0.022015831246972085 0.005321933776140213 0.0086173240467906
12 2.3

In [41]:
pred = torch.zeros(test_u.shape)
index = 0
# model = torch.load("model/friction_ep500")
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=1, shuffle=False)
first_output = 0

with torch.no_grad():
    for x, y in test_loader:
        test_l2 = 0
        if index == 0:
            first_output = y.clone()
        x, y = x.cuda(), y.cuda()
        out = model(x)
        out = y_normalizer.decode(out[:,:,:,0])
        pred[index] = out

        test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
        print(index, test_l2)

        # cp = plt.imshow(pred[index])
        # plt.colorbar(cp)
        # plt.show()

# for i in range(18):
#     cp = plt.imshow(abs(pred[0,:,:,i]-first_output[0,:,:,i]))
#     plt.colorbar(cp)
#     plt.show()
#     print(i)
    # cp = plt.imshow(first_output[0,:,:,i])
    # plt.colorbar(cp)
    # plt.show()

# cp = plt.imshow(first_output[0,:,:,0])
# plt.colorbar(cp)
# plt.show()

# cp = plt.imshow(pred[0,:,:,0])
# plt.colorbar(cp)
# plt.show()

# cp = plt.imshow(abs(pred[0,:,:,0]-first_output[0,:,:,0]))
# plt.colorbar(cp)
# plt.show()

# cp = plt.imshow(pred[0,:,:,9])
# plt.colorbar(cp)
# plt.show()
# cp = plt.imshow(pois_output[0,:,:,0])
# plt.colorbar(cp)
# plt.show()

# cp = plt.imshow(abs(pred[index][:,:,0] - pois_output[0,:,:,0]))
# plt.colorbar(cp)
# plt.show()


0 0.017842374742031097
0 0.014301486313343048
0 0.010149260982871056
0 0.005331197753548622
0 0.003250960260629654
0 0.002451823791489005
0 0.001721478532999754
0 0.001687671523541212
0 0.0018071928061544895
0 0.0019339019199833274
0 0.002026800299063325
0 0.0020760868210345507
0 0.0020523066632449627
0 0.0019202758558094501
0 0.0016626983415335417
0 0.0012704693945124745
0 0.0010023340582847595
0 0.0017700431635603309
0 0.003229493508115411
0 0.0014781467616558075
0 0.022783346474170685
0 0.01986577920615673
0 0.016513710841536522
0 0.012596605345606804
0 0.00796829629689455
0 0.0030615695286542177
0 0.002664623549208045
0 0.0019163703545928001
0 0.0020031898748129606
0 0.0021730782464146614
0 0.0023188774939626455
0 0.0024224903900176287
0 0.002479872666299343
0 0.0024788177106529474
0 0.002400450175628066
0 0.0022244518622756004
0 0.0018976236460730433
0 0.001454404555261135
0 0.0012693310854956508
0 0.0022861245088279247
0 0.027132775634527206
0 0.024665886536240578
0 0.02188719250